<a href="https://www.kaggle.com/code/emmermarcell/rag-pipeline-on-the-wikipedia-dataset?scriptVersionId=159749080" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Implementing a RAG pipeline on the Wikipedia dataset

Since the dataset is quite large I will make use of the memory mapping between the RAM and the filesystems storage done by the the [Hugging Face Datasets library][1]. Under the hood, it utilizes the Apache Arrow memory format and pyarrow library. Unfortunately the [`wikipedia`][8] dataset is not streamable so I stick to iterating through it.

The embedding of the Wikipedia articles are done with the [`all-MiniLM-L6-v2`][4] model from the [Sentence-Transformers][5] library.  The strings are embedded into a $384$ dimensional vector space where a similarity search is performed by the `faiss.IndexFlatL2` index based on their Euclidean (L2) distance.

The notebook runs on 2xT4 GPUs that Kaggle provides.
A great resource for training faiss on multiple GPUs can be found on the [faiss github site][2]. Furthermore, for computing embeddings on multiple GPUs I reference the [Sentence-Transformers github site][3].

After the embedding and ranking of athe article chunks, I employ the [`distilbert-base-cased-distilled-squad`][9] Q&A pipeline, a fine-tuned version of the [`DistilBERT-base-cased`][10] model using (a second step of) knowledge distillation on the [`SQuAD v1.1`][11] dataset.

I used the following articles as a starting point for implementing a RAG pipeline:

* [Akriti Upadhyay - Implementing RAG with Langchain and Hugging Face][6]

* [Vladimir Blagojevic - Ask Wikipedia ELI5-like Questions Using Long-Form Question Answering on Haystack][7]

[1]: https://huggingface.co/learn/nlp-course/chapter5/4?fw=pt
[2]: https://github.com/facebookresearch/faiss/blob/main/tutorial/python/5-Multiple-GPUs.py
[3]: https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/computing-embeddings/computing_embeddings_multi_gpu.py
[4]: https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
[5]: https://www.sbert.net/
[6]: https://medium.com/international-school-of-ai-data-science/implementing-rag-with-langchain-and-hugging-face-28e3ea66c5f7q=implementing+rag+with+langchain+and+huggingface&oq=implementing+rag+with+langchain+and+huggingface&gs_lcrp=EgZjaHJvbWUyBggAEEUYOTIKCAEQABiABBiiBDIKCAIQABiABBiiBNIBCDc2MzFqMGo3qAIAsAIA&client=ubuntu-chr&sourceid=chrome&ie=UTF-8
[7]: https://towardsdatascience.com/ask-wikipedia-eli5-like-questions-using-long-form-question-answering-on-haystack-32cf1ca6c00e
[8]: https://huggingface.co/datasets/wikipedia
[9]: https://huggingface.co/distilbert-base-cased-distilled-squad
[10]: https://huggingface.co/distilbert-base-cased
[11]: https://huggingface.co/datasets/squad

In [1]:
!pip install sentence_transformers
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 963.6 kB/s eta 0:00:00 0:00:01m
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=e1f5e4eba8284c5d3de8bafdfec9741c1ef171681c5ca89f5a0f245456cf7b1b
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 14.3 MB/s eta 0:00:0000:0100:01


In [2]:
import gc    # Garbage collector
import logging
import time
import re
from tqdm.auto import tqdm
from nltk.tokenize import word_tokenize
from concurrent.futures import ProcessPoolExecutor
import numpy as np
from datasets import load_dataset
from transformers import pipeline
from sentence_transformers import SentenceTransformer, LoggingHandler
import faiss


logging.basicConfig(
    format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO, handlers=[LoggingHandler()]
)


# Ensure you have a GPU available
ngpus = faiss.get_num_gpus()
print("number of GPUs:", ngpus)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


number of GPUs: 2


The `all-MiniLM-L6-v2` model can handle a maximum sequence size of 256 words. As Wikipedia articles are often longer, we use the `process_strings_and_combine_parallel` function to preprocess the data. This function takes a list of strings (representing Wikipedia articles) and returns another list of strings. It breaks up each string into chunks of words, and the size of each chunk is determined by the `chunk_size` parameter. The text processing is done with the `ProcessPoolExecutor`method from `concurrent.futures` that uses a pool of processes to execute calls asynchronously.

In [3]:
def combine_into_chunks(words_list, chunk_size):
    # Generate chunks of the specified size from the words list
    chunks = [words_list[i:i + chunk_size] for i in range(0, len(words_list), chunk_size)]
    # Join each chunk into a single string and return the list of chunks
    return [' '.join(chunk) for chunk in chunks]

def process_string(text, chunk_size):
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    words_list = word_tokenize(cleaned_text)
    return combine_into_chunks(words_list, chunk_size)

def process_strings_and_combine_parallel(string_list, chunk_size):
    result_chunks = []

    with ProcessPoolExecutor() as executor:
        # Pass chunk_size as an argument to the process_string function
        chunks_list = list(executor.map(process_string, string_list, [chunk_size] * len(string_list)))

    for chunks in chunks_list:
        result_chunks.extend(chunks)

    return result_chunks

In [4]:
# Important, you need to shield your code with if __name__. Otherwise, CUDA runs into issues when spawning new processes.
if __name__ == "__main__":
    # Define the sentence transformer model
    model_name = "all-MiniLM-L6-v2"
    model = SentenceTransformer(model_name)
    embedding_dim = model.get_sentence_embedding_dimension()    # Get the embedding dimension
    max_seq_len = model.max_seq_length    # Maximum sequence length in words
    print(f'The embedding dimension of the all-MiniLM-L6-v2 model is {embedding_dim}.')
    print(f"Max sequence lenght of the {model_name} model is {max_seq_len}.")
    
    # Initialize a FAISS index (for CPU)
    cpu_index = faiss.IndexFlatL2(embedding_dim)

    # Initialize GPU resources for FAISS
    gpu_index = faiss.index_cpu_to_all_gpus(  # build the index
        cpu_index
    )
    
    # Loading in the Wikipedia dataset
    wiki_dataset = load_dataset("wikipedia", "20220301.en", split='train[:5000]')
    print(f'Lenght of the Wikipedia dataset is {len(wiki_dataset)} articles.')
    
    # Process the articles into a list of 256 word strings
    start_time = time.time()
    processed_articles = process_strings_and_combine_parallel(wiki_dataset['text'], chunk_size=max_seq_len)
    end_time = time.time()
    print(f"Sequential processing of the dataset took {end_time - start_time:.2f} seconds.")
        
        
    # Start the multi-process pool on all available CUDA devices
    pool = model.start_multi_process_pool()
    
    # Batch processing with tqdm progress bar
    batch_size = 1024  # Define batch size based on the system's memory capacity
    total_batches = len(processed_articles) // batch_size + (0 if len(processed_articles) % batch_size == 0 else 1)

    for i in tqdm(range(0, len(processed_articles), batch_size), total=total_batches, desc="Processing Batches"):
        # Take the next batch of articles
        batch_texts = processed_articles[i:i + batch_size]
        # Compute the embeddings using the multi-process pool
        batch_embeddings = model.encode_multi_process(batch_texts, pool)
        # Add embeddings to the GPU index
        gpu_index.add(batch_embeddings)
        
        # Memory management
        del batch_embeddings, batch_texts
        gc.collect()
        
    # Function to search for relevant articles using GPU
    def search_wiki_articles(question):
        question_embedding = model.encode_multi_process(question, pool)
        distances, indices = gpu_index.search(question_embedding, k=3)
        return [processed_articles[i] for i in indices[0]]

    # State business questions
    questions = [
        'What services does KPMG offer to its clients?',
        'What are the key considerations when assessing internal controls during an audit?',
        'How do you stay updated on changes in tax laws and regulations affecting clients?',
        "What steps do you take to understand a client's business before initiating a consulting project?",
        'What due diligence processes are crucial for evaluating the financial health of a potential acquisition?',
    ]
    
    relevant_article_chunks = [search_wiki_articles(question) for question in questions]
    
    # Example
    print(f'Question:\n{questions[0]}')
    print(f'Relevant Wikipedia article chunks:\n{relevant_article_chunks[0]}')
    
    # Optional: Stop the processes in the pool
    model.stop_multi_process_pool(pool)
    
    # (Optional) Save the processed dataset as JSON file and the faiss index
    """
    import json
    
    with open("processed_wiki_articles.json", 'w') as file:
        json.dump(processed_articles, file)
        
    faiss.write_index(gpu_index, 'Wikipedia_FlatL2.index')
    """

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/20.3G [00:00<?, ?B/s]

Dataset wikipedia downloaded and prepared to /root/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559. Subsequent calls will reuse this data.
Lenght of the Wikipedia dataset is 5000 articles.
Sequential processing of the dataset took 52.18 seconds.


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Processing Batches:   0%|          | 0/67 [00:00<?, ?it/s]

Question:
What services does KPMG offer to its clients?
Relevant Wikipedia article chunks:
['city', 'economic conditions on March 5 2011 WZZM resumed the Saturday and Sunday morning newscasts after a twoyear absence airing for two hours from 6 to 8 am on both days They now also offer a onehour 9 am Newscast on Sunday mornings after GMA In late 2009 WZZM became the third television station in West Michigan to begin broadcasting its local newscasts in widescreen standard definition WWMT was the last major station in West Michigan with 43 standard definition newscasts until April 16 2011 when it became the second station in the market to upgrade to full high definition newscasts In June 2010 WZZM rehired Brent Ashcroft who had left the station twelve years prior to become sports director at Fox affiliate WXMI channel 17 On December 3 2011 WZZM became the fourth and final television station in West Michigan to begin broadcasting its local newscasts in high definition WZZMTV focuses its new

The maximum length of tokens the we can feed to the tokenizer before truncation is $512$. Therefore it is no use to search for the k=3 kNN of article chunks for a given question using the faiss index.

In [6]:
# Load the Q&A pipeline
qa_model_name = 'distilbert-base-cased-distilled-squad'
question_answerer = pipeline('question-answering', model=qa_model_name, tokenizer=qa_model_name)
print(f'The maximum length of tokens the we can feed to the tokenizer before truncation is {question_answerer.tokenizer.model_max_length}.')

# Function to perform question-answering given a question and a list of documents
def answer_question(question, article_chunks):
    # Combine the article chunks into a single string
    context = ' '.join(article_chunks)

    # Perform question-answering
    result = question_answerer(question=question, context=context)

    return result['answer']

# Answer the questions
results = [answer_question(questions[idx], relevant_article_chunks[idx]) for idx in range(len(questions))]

#for result in results:
#    print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")
    
for idx in range(len(questions)):
    print(f'Question:\n{questions[idx]}')
    print(f'Answer:\n{results[idx]}')
    print('='*30)

The maximum length of tokens the we can feed to the tokenizer before truncation is 512.
Question:
What services does KPMG offer to its clients?
Answer:
11 full power stations
Question:
What are the key considerations when assessing internal controls during an audit?
Answer:
accuracy of the by Nielsen Media Research 14
Question:
How do you stay updated on changes in tax laws and regulations affecting clients?
Answer:
median of a population or thresholding parameter
Question:
What steps do you take to understand a client's business before initiating a consulting project?
Answer:
longrunning late night duo Big Chuck and Lil John Radio Cleveland
Question:
What due diligence processes are crucial for evaluating the financial health of a potential acquisition?
Answer:
Nielsen Media Research 14


## Evaluation of the pipeline

For evaluation a Q&A pipeline, one an use the [Official Evaluation Script][1] of the [SQuAD v2.0][2] dataset. I inclued a part of the script that I can evaluate the Exact and the F1 score of a pipeline on this dataset.

[1]: https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/
[2]: https://rajpurkar.github.io/SQuAD-explorer/

In [ ]:
"""Official evaluation script for SQuAD version 2.0.

In addition to basic functionality, we also compute additional statistics and
plot precision-recall curves if an additional na_prob.json file is provided.
This file is expected to map question ID's to the model's predicted probability
that a question is unanswerable.
"""
import argparse
import collections
import json
import numpy as np
import os
import re
import string
import sys

OPTS = None

def parse_args():
    parser = argparse.ArgumentParser('Official evaluation script for SQuAD version 2.0.')
    parser.add_argument('data_file', metavar='data.json', help='Input data JSON file.')
    parser.add_argument('pred_file', metavar='pred.json', help='Model predictions.')
    parser.add_argument('--out-file', '-o', metavar='eval.json',
                        help='Write accuracy metrics to file (default is stdout).')
    parser.add_argument('--na-prob-file', '-n', metavar='na_prob.json',
                        help='Model estimates of probability of no answer.')
    parser.add_argument('--na-prob-thresh', '-t', type=float, default=1.0,
                        help='Predict "" if no-answer probability exceeds this (default = 1.0).')
    parser.add_argument('--out-image-dir', '-p', metavar='out_images', default=None,
                        help='Save precision-recall curves to directory.')
    parser.add_argument('--verbose', '-v', action='store_true')
    if len(sys.argv) == 1:
        parser.print_help()
        sys.exit(1)
    return parser.parse_args()

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1